<a href="https://colab.research.google.com/github/SeekingNirvaana/Data-Analysis-Visualisation/blob/main/MIT_Air_Quality_DS_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install cdsapi
clear_output()

In [ ]:
!pip install cartopy
clear_output()

In [ ]:
# CDS API
import cdsapi
import os

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from matplotlib.colors import ListedColormap

from datetime import datetime

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import seaborn as sns

In [ ]:
URL = 'https://ads.atmosphere.copernicus.eu/api/v2'
KEY = '10182:abaca44d-2fcd-46a2-8cd7-75892a2ac515'

In [ ]:
import cdsapi
c = cdsapi.Client(url=URL, key=KEY)

In [ ]:
c.retrieve(
    'cams-global-reanalysis-eac4',
    {
  "area": [
    37.1,
    68.11,
    8.06,
    97.41
  ],
  "date": "2020-01-01/2020-12-31",
  "format": "netcdf",
  "time": [
    "00:00",
    "03:00",
    "06:00",
    "09:00",
    "12:00",
    "15:00",
    "18:00",
    "21:00"
  ],
  "variable": [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_dewpoint_temperature",
    "2m_temperature",
    "black_carbon_aerosol_optical_depth_550nm",
    "dust_aerosol_optical_depth_550nm",
    "mean_sea_level_pressure",
    "organic_matter_aerosol_optical_depth_550nm",
    "particulate_matter_2.5um",
    "surface_pressure",
    "total_aerosol_optical_depth_1240nm",
    "total_aerosol_optical_depth_469nm",
    "total_aerosol_optical_depth_550nm",
    "total_aerosol_optical_depth_670nm",
    "total_aerosol_optical_depth_865nm",
    "total_column_carbon_monoxide",
    "total_column_nitrogen_dioxide",
    "total_column_nitrogen_monoxide",
    "total_column_sulphur_dioxide",
    "total_column_water_vapour"
  ]
},
    'download.nc')

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2022-06-24 12:28:02,754 INFO Welcome to the CDS
2022-06-24 12:28:02,757 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2022-06-24 12:28:02,980 INFO Request is completed
2022-06-24 12:28:02,982 INFO Downloading https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data7/adaptor.mars.internal-1655551464.323058-13271-7-6dab7463-3b97-

Result(content_length=182725760,content_type=application/x-netcdf,location=https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data7/adaptor.mars.internal-1655551464.323058-13271-7-6dab7463-3b97-4155-a8bb-41cad4da2f2f.nc)

In [ ]:
xr_data = xr.open_dataset('download.nc')
xr_data

<xarray.Dataset>
Dimensions:    (longitude: 40, latitude: 39, time: 2928)
Coordinates:
  * longitude  (longitude) float32 68.11 68.86 69.61 70.36 ... 95.86 96.61 97.36
  * latitude   (latitude) float32 36.56 35.81 35.06 34.31 ... 9.56 8.81 8.06
  * time       (time) datetime64[ns] 2020-01-01 ... 2020-12-31T21:00:00
Data variables: (12/20)
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    d2m        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    bcaod550   (time, latitude, longitude) float32 ...
    duaod550   (time, latitude, longitude) float32 ...
    ...         ...
    aod865     (time, latitude, longitude) float32 ...
    tcco       (time, latitude, longitude) float32 ...
    tcno2      (time, latitude, longitude) float32 ...
    tc_no      (time, latitude, longitude) float32 ...
    tcso2      (time, latitude, longitude) float32 ...
    tcwv       (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-06-18 11:26:53 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

In [ ]:
print(xr_data.time)

<xarray.DataArray 'time' (time: 2928)>
array(['2020-01-01T00:00:00.000000000', '2020-01-01T03:00:00.000000000',
       '2020-01-01T06:00:00.000000000', ..., '2020-12-31T15:00:00.000000000',
       '2020-12-31T18:00:00.000000000', '2020-12-31T21:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 ... 2020-12-31T21:00:00
Attributes:
    long_name:  time


In [ ]:
df = xr_data.to_dataframe()

In [ ]:
df.head(50)

u10       v10         d2m  \
longitude latitude  time                                                  
68.110001 36.560001 2020-01-01 00:00:00  0.232746  0.440074  272.845428   
                    2020-01-01 03:00:00  0.193819  1.426527  273.068054   
                    2020-01-01 06:00:00  0.307245  1.232609  274.525452   
                    2020-01-01 09:00:00  0.543493  0.036780  275.641235   
                    2020-01-01 12:00:00  1.411302 -0.375647  275.185699   
                    2020-01-01 15:00:00  1.647550  1.376642  275.558044   
                    2020-01-01 18:00:00  0.885784  1.614825  275.454407   
                    2020-01-01 21:00:00  0.319997  1.515055  275.206177   
                    2020-01-02 00:00:00 -0.334384  0.901683  274.000854   
                    2020-01-02 03:00:00 -0.100149  1.089278  274.200470   
                    2020-01-02 06:00:00  0.289123  0.541951  275.852356   
                    2020-01-02 09:00:00  0.217980 -0.646149  275.879211   
                    2020-01-02 12:00:00  0.224021 -0.872387  278.228485   
                    2020-01-02 15:00:00 -0.390762  0.578486  277.547760   
                    2020-01-02 18:00:00 -0.241764  0.376840  277.058960   
                    2020-01-02 21:00:00  0.648865  0.808940  276.983490   
                    2020-01-03 00:00:00  0.721350  0.171680  277.221466   
                    2020-01-03 03:00:00  0.852226  1.497490  277.248352   
                    2020-01-03 06:00:00  0.870348  0.541248  277.056396   
                    2020-01-03 09:00:00  1.060957 -1.067008  277.216339   
                    2020-01-03 12:00:00  0.507250 -1.302379  276.240051   
                    2020-01-03 15:00:00 -0.065249  1.038691  275.153717   
                    2020-01-03 18:00:00  0.186436  1.597260  274.121124   
                    2020-01-03 21:00:00  0.026700  1.292330  273.453217   
                    2020-01-04 00:00:00  0.464967  1.531917  272.437256   
                    2020-01-04 03:00:00  0.420671  1.545267  272.542175   
                    2020-01-04 06:00:00  0.509264  0.364193  274.162079   
                    2020-01-04 09:00:00  0.122004 -1.629792  275.523499   
                    2020-01-04 12:00:00  0.607253 -1.324863  276.028931   
                    2020-01-04 15:00:00  1.187806  0.234211  272.745605   
                    2020-01-04 18:00:00  0.460269  0.349438  271.779541   
                    2020-01-04 21:00:00  0.011263  0.420401  271.820496   
                    2020-01-05 00:00:00  0.377716  0.981077  273.546600   
                    2020-01-05 03:00:00  0.046835  1.248066  273.157623   
                    2020-01-05 06:00:00 -0.159211  0.736572  275.175476   
                    2020-01-05 09:00:00 -1.138433 -0.088283  275.405792   
                    2020-01-05 12:00:00  0.301204 -0.767699  274.910614   
                    2020-01-05 15:00:00  1.765003 -0.026454  274.769867   
                    2020-01-05 18:00:00  1.237472  0.674743  274.568970   
                    2020-01-05 21:00:00  0.991827  1.385074  274.813354   
                    2020-01-06 00:00:00  1.264318  1.412475  274.799286   
                    2020-01-06 03:00:00  1.469693  1.207315  275.703918   
                    2020-01-06 06:00:00  1.371033  0.514550  275.313660   
                    2020-01-06 09:00:00  2.060314 -0.624369  275.254822   
                    2020-01-06 12:00:00  1.630100 -0.608209  275.792206   
                    2020-01-06 15:00:00  1.920041 -0.477525  275.325195   
                    2020-01-06 18:00:00  1.848227 -0.577294  273.670715   
                    2020-01-06 21:00:00  1.584461 -0.488064  272.781433   
                    2020-01-07 00:00:00  1.319353 -1.061387  272.011139   
                    2020-01-07 03:00:00  1.212639  0.128118  271.654144   

                                                t2m  bcaod550  duaod550  \
longitude latitude  time                                              